In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz

     |████████████████████████████████| 827.9 MB 4.5 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=d493f26268757e3c278d7441fa8cd419f4a7a09f8ba39d66efe22f07b8ca1091
  Stored in directory: /root/.cache/pip/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 2.1 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf
 教授资格证身份证.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529-a0611_UK		       dataframe_result_csv_EU
 a0529-a0611_US		       dataframe_result_csv_UK
 a0608.csv		       Github
 a0609.csv		       heatmap529_UK.ipynb
 ccc_compiled.csv	       heatmap530.ipynb
 csv			       Heat_Map_Online_language.ipynb
 csv.zip		       __MACOSX
 dataframe_csv		       mapping_UK
 dataframe_csv_EU	       mapping_US
 dataframe_csv_UK	       non_violant1.csv
 dataframe_result_529_UK.csv   non_violant2.csv
 dataframe_result_530_UK.csv   UK
 dataframe_result_531_UK.csv   violant.csv
 dataframe_result_csv	      '新建 Microsoft PowerPoint 演示文稿.pptx'


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [21]:
ct_path_601 = 'UK/06_05.csv'

In [22]:
ct = pd.read_csv(ct_path_601)

In [23]:
ct.shape[1]

47

In [24]:
columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [25]:
#len(columns1)
len(columns2)

47

In [26]:
ct.columns=columns2

In [27]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,48597654813_10158381384639814,Facebook,2020-06-05 23:59:48,2021-08-31 05:39:29,link,Black Lives Matter protest in Ada honors Antho...,kxii.com,-,Hundreds of people came together for a Black L...,1.0,https://www.kxii.com/content/news/Black-Lives-...,https://www.facebook.com/48597654813/posts/101...,125613,-2.015625,1.0,25,11,7,16,0,3,0,2,0,0,24,14,35,5,8,14,16,13,0,4,24653,KXII-TV,channel12,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Texoma's Breaking News & Weather Authority ser...,True,2009-02-04 21:03:03,en,0,24653|10158381384639814
1,57406800343_10163759708125344,Facebook,2020-06-05 23:59:38,2021-07-27 09:14:41,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,-,Check out the latest episode of the In the Zon...,1.0,https://www.kxii.com/content/news/Black-Lives-...,https://www.facebook.com/groups/57406800343/pe...,1373,-0.666667,1.0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,8520182,Intertwine Chicago,channel12,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,Texoma's Breaking News & Weather Authority ser...,False,2009-02-04 21:03:03,en,0,8520182|10163759708125344
2,102051553222110_2992368327523737,Facebook,2020-06-05 23:59:37,2021-07-08 21:08:55,photo,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...",Read Linda Silas' full statement here: https:/...,2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/102051553222110/posts...,8927,2.681818,1.0,44,10,0,5,0,0,0,0,0,0,7,5,2,3,1,1,2,1,0,2,190096,Canadian Federation of Nurses Unions (CFNU),NursesUnions,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,CA,The official Facebook page of the Canadian Fed...,False,2011-06-01 18:46:23,en,0,190096|2992368327523737
3,322314941151928_3245155888867804,Facebook,2020-06-05 23:59:32,2021-08-24 19:47:41,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...","Family, friends, neighbors; We wanted to take ...",2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/322314941151928/posts...,2926,5.915888,1.0,201,208,33,190,1,0,0,0,0,28,52,23,8,10,2,1,10,1,0,8,2904250,The Coeur d'Alene Tribe,cdatribe,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Welcome to the official page for the Schitsu'u...,False,2012-02-08 21:52:13,en,0,2904250|3245155888867804
4,146265275458081_3032237670194146,Facebook,2020-06-05 23:59:26,2021-06-28 20:32:20,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...",Check out the latest episode of the In the Zon...,2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/groups/14626527545808...,1188,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,2,3,2,0,2,0,0,0,1,10227323,Y.O.U. Young Opulent & Unique,cdatribe,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,Welcome to the official page for the Schitsu'u...,False,2012-02-08 21:52:13,en,0,10227323|3032237670194146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35409,460670040801275_14339257

In [28]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [29]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [30]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [31]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [32]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [33]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [34]:
ct['NER-msg'] = ct['message'].apply(ner)

In [35]:
ct['NER-desc'] = ct['description'].apply(ner)

In [36]:
ct['NER-title'] = ct['title'].apply(ner)

In [37]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,{'Ada': 1},{}
1,{},{}
2,{},{'Canada': 2}
3,{'Minneapolis': 2},{'Canada': 2}
4,{},{'Canada': 2}


In [38]:
ct.to_csv('a0605.csv')

In [ ]:
!ls

In [86]:
ct=pd.read_csv('a0605.csv',lineterminator='\n',encoding='ISO-8859-1')

In [42]:
###
#ct=pd.read_csv('a0601.csv')

In [ ]:
ct

In [88]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [45]:
#ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [89]:
###
ct_ner = ct

In [90]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,{'Ada': 1},{},{}
1,{},{},{}
2,{},{'Canada': 2},{}
3,{'Minneapolis': 2},{'Canada': 2},{}
4,{},{'Canada': 2},{}


In [98]:
ct_ner['NER-msg'][0].keys()

dict_keys(['Ada'])

In [101]:
[*ct_ner['NER-msg'][0]]

['Ada']

In [97]:
list(ct_ner['NER-msg'][0])

TypeError: ignored

In [104]:
for i in range(len(ct_ner['NER-msg'])):
  ct_ner['NER-msg'][i]=[*ct_ner['NER-msg'][i]]
ct_ner['NER-msg']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0                     [Ada]
1                        []
2                        []
3             [Minneapolis]
4                        []
                ...        
35409                    []
35410                    []
35411                    []
35412    [America, Lebanon]
35413                    []
Name: NER-msg, Length: 35414, dtype: object

In [105]:
for i in range(len(ct_ner['NER-desc'])):
  ct_ner['NER-desc'][i]=[*ct_ner['NER-desc'][i]]
ct_ner['NER-desc']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0                                  []
1                                  []
2                            [Canada]
3                            [Canada]
4                            [Canada]
                     ...             
35409                              []
35410                              []
35411    [IOWA CITY, Iowa, Iowa City]
35412              [America, Lebanon]
35413              [America, Lebanon]
Name: NER-desc, Length: 35414, dtype: object

In [106]:
for i in range(len(ct_ner['NER-title'])):
  ct_ner['NER-title'][i]=[*ct_ner['NER-title'][i]]
ct_ner['NER-title']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0                 []
1                 []
2                 []
3                 []
4                 []
            ...     
35409             []
35410             []
35411    [Iowa City]
35412             []
35413             []
Name: NER-title, Length: 35414, dtype: object

In [91]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

TypeError: ignored

In [69]:
ct_ner['NER-msg']

0                          {'Ada': 1}
1                                  {}
2                                  {}
3                  {'Minneapolis': 2}
4                                  {}
                     ...             
35409                              {}
35410                              {}
35411                              {}
35412    {'America': 1, 'Lebanon': 1}
35413                              {}
Name: NER-msg, Length: 35414, dtype: object

In [107]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

In [108]:
ct_ner['NER']

0                                          [Ada]
1                                             []
2                                       [Canada]
3                          [Minneapolis, Canada]
4                                       [Canada]
                          ...                   
35409                                         []
35410                                         []
35411    [IOWA CITY, Iowa, Iowa City, Iowa City]
35412       [America, Lebanon, America, Lebanon]
35413                         [America, Lebanon]
Name: NER, Length: 35414, dtype: object

In [109]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

TypeError: ignored

In [110]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

In [111]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER
0,0,48597654813_10158381384639814,Facebook,2020-06-05 23:59:48,2021-08-31 05:39:29,link,Black Lives Matter protest in Ada honors Antho...,kxii.com,-,Hundreds of people came together for a Black L...,1.0,https://www.kxii.com/content/news/Black-Lives-...,https://www.facebook.com/48597654813/posts/101...,125613,-2.015625,1.0,25,11,7,16,0,3,0,2,0,0,24,14,35,5,8,14,16,13,0,4,24653,KXII-TV,channel12,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Texoma's Breaking News & Weather Authority ser...,True,2009-02-04 21:03:03,en,0,24653|10158381384639814,[Ada],[],[],[ada]
1,1,57406800343_10163759708125344,Facebook,2020-06-05 23:59:38,2021-07-27 09:14:41,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,-,Check out the latest episode of the In the Zon...,1.0,https://www.kxii.com/content/news/Black-Lives-...,https://www.facebook.com/groups/57406800343/pe...,1373,-0.666667,1.0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,8520182,Intertwine Chicago,channel12,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,Texoma's Breaking News & Weather Authority ser...,False,2009-02-04 21:03:03,en,0,8520182|10163759708125344,[],[],[],[]
2,2,102051553222110_2992368327523737,Facebook,2020-06-05 23:59:37,2021-07-08 21:08:55,photo,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...",Read Linda Silas' full statement here: https:/...,2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/102051553222110/posts...,8927,2.681818,1.0,44,10,0,5,0,0,0,0,0,0,7,5,2,3,1,1,2,1,0,2,190096,Canadian Federation of Nurses Unions (CFNU),NursesUnions,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,CA,The official Facebook page of the Canadian Fed...,False,2011-06-01 18:46:23,en,0,190096|2992368327523737,[],[Canada],[],[canada]
3,3,322314941151928_3245155888867804,Facebook,2020-06-05 23:59:32,2021-08-24 19:47:41,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...","Family, friends, neighbors; We wanted to take ...",2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/322314941151928/posts...,2926,5.915888,1.0,201,208,33,190,1,0,0,0,0,28,52,23,8,10,2,1,10,1,0,8,2904250,The Coeur d'Alene Tribe,cdatribe,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Welcome to the official page for the Schitsu'u...,False,2012-02-08 21:52:13,en,0,2904250|3245155888867804,[Minneapolis],[Canada],[],"[minneapolis, canada]"
4,4,146265275458081_3032237670194146,Facebook,2020-06-05 23:59:26,2021-06-28 20:32:20,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...",Check out the latest episode of the In the Zon...,2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/groups/14626527545808...,1188,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,2,3,2,0,2,0,0,0,1,10227323,Y.O.U. Young Opulent & Unique,cdatribe,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,Welcome to the official page for the Schitsu'u...,False,2012-02-08 21:52:13,en,0,10227323|3032237670194146,[],[Canada],[],[canada]
...,...,...,...,...,...,...,...,...,...,..

In [112]:
ct_ner['NER-1']=ct_ner['NER']

In [113]:
ct_ner['NER-1'][0]

['ada']

In [114]:
!pip install -q geopy
!pip install -q pyproj

# basic utilities
import json
import requests
import pandas as pd
import pprint
import time

# reverse geocoding utilities
from geopy.geocoders import Nominatim

# geography projection utilities
import pyproj
from shapely.ops import transform
from shapely.geometry import Point
from functools import partial

     |████████████████████████████████| 6.3 MB 10.3 MB/s 


In [115]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [116]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,48597654813_10158381384639814,Facebook,2020-06-05 23:59:48,2021-08-31 05:39:29,link,Black Lives Matter protest in Ada honors Antho...,kxii.com,-,Hundreds of people came together for a Black L...,1.0,https://www.kxii.com/content/news/Black-Lives-...,https://www.facebook.com/48597654813/posts/101...,125613,-2.015625,1.0,25,11,7,16,0,3,0,2,0,0,24,14,35,5,8,14,16,13,0,4,24653,KXII-TV,channel12,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Texoma's Breaking News & Weather Authority ser...,True,2009-02-04 21:03:03,en,0,24653|10158381384639814,[Ada],[],[],[ada],[ada]
1,1,57406800343_10163759708125344,Facebook,2020-06-05 23:59:38,2021-07-27 09:14:41,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,-,Check out the latest episode of the In the Zon...,1.0,https://www.kxii.com/content/news/Black-Lives-...,https://www.facebook.com/groups/57406800343/pe...,1373,-0.666667,1.0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,8520182,Intertwine Chicago,channel12,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,Texoma's Breaking News & Weather Authority ser...,False,2009-02-04 21:03:03,en,0,8520182|10163759708125344,[],[],[],[],[]
2,2,102051553222110_2992368327523737,Facebook,2020-06-05 23:59:37,2021-07-08 21:08:55,photo,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...",Read Linda Silas' full statement here: https:/...,2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/102051553222110/posts...,8927,2.681818,1.0,44,10,0,5,0,0,0,0,0,0,7,5,2,3,1,1,2,1,0,2,190096,Canadian Federation of Nurses Unions (CFNU),NursesUnions,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,CA,The official Facebook page of the Canadian Fed...,False,2011-06-01 18:46:23,en,0,190096|2992368327523737,[],[Canada],[],[canada],[canada]
3,3,322314941151928_3245155888867804,Facebook,2020-06-05 23:59:32,2021-08-24 19:47:41,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...","Family, friends, neighbors; We wanted to take ...",2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/322314941151928/posts...,2926,5.915888,1.0,201,208,33,190,1,0,0,0,0,28,52,23,8,10,2,1,10,1,0,8,2904250,The Coeur d'Alene Tribe,cdatribe,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Welcome to the official page for the Schitsu'u...,False,2012-02-08 21:52:13,en,0,2904250|3245155888867804,[Minneapolis],[Canada],[],"[minneapolis, canada]","[minneapolis, canada]"
4,4,146265275458081_3032237670194146,Facebook,2020-06-05 23:59:26,2021-06-28 20:32:20,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...",Check out the latest episode of the In the Zon...,2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/groups/14626527545808...,1188,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,2,3,2,0,2,0,0,0,1,10227323,Y.O.U. Young Opulent & Unique,cdatribe,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,Welcome to the official page for the Schitsu'u...,False,2012-02-08 21:52:13,en,0,10227323|3032237670194146,[],[Canada],[],[ca

In [117]:
%cd mapping_UK

/content/gdrive/My Drive/2021-online-language-offline-impact-main/mapping_UK


In [118]:
!ls

dataframe0529_post.csv	dataframe0607_post.csv	    dataframe_mapping_0603.csv
dataframe0530_post.csv	dataframe0608_post.csv	    dataframe_mapping_0604.csv
dataframe0531_post.csv	dataframe0609_post.csv	    dataframe_mapping_0605.csv
dataframe0601_post.csv	dataframe0610_post.csv	    dataframe_mapping_0606.csv
dataframe0602_post.csv	dataframe_mapping_0529.csv  dataframe_mapping_0607.csv
dataframe0603_post.csv	dataframe_mapping_0530.csv  dataframe_mapping_0608.csv
dataframe0604_post.csv	dataframe_mapping_0531.csv  dataframe_mapping_0609.csv
dataframe0605_post.csv	dataframe_mapping_0601.csv  dataframe_mapping_0610.csv
dataframe0606_post.csv	dataframe_mapping_0602.csv  dataframe_mapping_0611.csv


In [119]:
ct_mapping=pd.read_csv('dataframe_mapping_0605.csv',encoding='ISO-8859-1')

In [120]:
len(ct_mapping['City'])

4651

In [121]:
len(ct_ner['NER-1'])

35414

In [122]:
for k in range(23900):
  list=[]
  for i in ct_ner['NER-1'][k]:
    for j in range(1538):
      if(i==ct_mapping['City'][j]):
        list.append(ct_mapping['Mapping'][j])
  ct_ner['NER-1'][k]=list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [123]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,48597654813_10158381384639814,Facebook,2020-06-05 23:59:48,2021-08-31 05:39:29,link,Black Lives Matter protest in Ada honors Antho...,kxii.com,-,Hundreds of people came together for a Black L...,1.0,https://www.kxii.com/content/news/Black-Lives-...,https://www.facebook.com/48597654813/posts/101...,125613,-2.015625,1.0,25,11,7,16,0,3,0,2,0,0,24,14,35,5,8,14,16,13,0,4,24653,KXII-TV,channel12,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Texoma's Breaking News & Weather Authority ser...,True,2009-02-04 21:03:03,en,0,24653|10158381384639814,[Ada],[],[],[ada],[]
1,1,57406800343_10163759708125344,Facebook,2020-06-05 23:59:38,2021-07-27 09:14:41,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,-,Check out the latest episode of the In the Zon...,1.0,https://www.kxii.com/content/news/Black-Lives-...,https://www.facebook.com/groups/57406800343/pe...,1373,-0.666667,1.0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,8520182,Intertwine Chicago,channel12,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,Texoma's Breaking News & Weather Authority ser...,False,2009-02-04 21:03:03,en,0,8520182|10163759708125344,[],[],[],[],[]
2,2,102051553222110_2992368327523737,Facebook,2020-06-05 23:59:37,2021-07-08 21:08:55,photo,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...",Read Linda Silas' full statement here: https:/...,2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/102051553222110/posts...,8927,2.681818,1.0,44,10,0,5,0,0,0,0,0,0,7,5,2,3,1,1,2,1,0,2,190096,Canadian Federation of Nurses Unions (CFNU),NursesUnions,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,CA,The official Facebook page of the Canadian Fed...,False,2011-06-01 18:46:23,en,0,190096|2992368327523737,[],[Canada],[],[canada],[Canada]
3,3,322314941151928_3245155888867804,Facebook,2020-06-05 23:59:32,2021-08-24 19:47:41,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...","Family, friends, neighbors; We wanted to take ...",2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/322314941151928/posts...,2926,5.915888,1.0,201,208,33,190,1,0,0,0,0,28,52,23,8,10,2,1,10,1,0,8,2904250,The Coeur d'Alene Tribe,cdatribe,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Welcome to the official page for the Schitsu'u...,False,2012-02-08 21:52:13,en,0,2904250|3245155888867804,[Minneapolis],[Canada],[],"[minneapolis, canada]","[Minneapolis, Hennepin County, Minnesota, 5541..."
4,4,146265275458081_3032237670194146,Facebook,2020-06-05 23:59:26,2021-06-28 20:32:20,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...",Check out the latest episode of the In the Zon...,2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/groups/14626527545808...,1188,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,2,3,2,0,2,0,0,0,1,10227323,Y.O.U. Young Opulent & Unique,cdatribe,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,Welcome to the official page for the Schitsu'u...,False,2012-02-08 21:52:13,en,0,10227323|3032237670

In [124]:
ct_ner['NER-1'][12]

['Denver, Denver County, Colorado, United States']

In [125]:
len(dict_a)

NameError: ignored

In [126]:
ct_ner.to_csv('dataframe0605_post.csv')